In [10]:
#new book
import pandas as pd
import tensorflow as tf
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df= pd.read_csv("/content/drive/MyDrive/Genre Classification Dataset/train_data.txt",sep=":::",names=[
    'slno','title','genre','plot'])

<ipython-input-2-3c25b41a4320>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df= pd.read_csv("/content/drive/MyDrive/Genre Classification Dataset/train_data.txt",sep=":::",names=[


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Preprocess the text
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Tokenize, remove stopwords, and lemmatize
    tokens = nltk.word_tokenize(text.lower())
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.isalnum()]
    return ' '.join(filtered_tokens)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
nltk.download('punkt')
df['processed_plot'] = df['plot'].apply(preprocess_text)

# Preprocess the genres
df['genre'] = df['genre'].apply(lambda x: x.split('|'))
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['genre'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
genres_encoded

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
#!wget http://nlp.stanford.edu/df/glove.6B.zip
#!unzip glove*.zip
# Load the pre-trained GloVe embeddings
embeddings_index = {}
with open('/content/drive/MyDrive/Genre Classification Dataset/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Tokenize and pad the text sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['processed_plot'])
sequences = tokenizer.texts_to_sequences(df['processed_plot'])
word_index = tokenizer.word_index
padded_sequences = pad_sequences(sequences, maxlen=300)

# Create the embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)


# Define the LSTM model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], input_length=300, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(mlb.classes_), activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, genres_encoded, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
678/678 [==============================] - 174s 253ms/step - loss: 0.1181 - accuracy: 0.3921 - val_loss: 0.1050 - val_accuracy: 0.4457
Epoch 2/10
678/678 [==============================] - 170s 250ms/step - loss: 0.1007 - accuracy: 0.4555 - val_loss: 0.0960 - val_accuracy: 0.4746
Epoch 3/10
678/678 [==============================] - 169s 250ms/step - loss: 0.0947 - accuracy: 0.4851 - val_loss: 0.0914 - val_accuracy: 0.5002
Epoch 4/10
678/678 [==============================] - 169s 249ms/step - loss: 0.0905 - accuracy: 0.5044 - val_loss: 0.0889 - val_accuracy: 0.5134
Epoch 5/10
678/678 [==============================] - 169s 250ms/step - loss: 0.0873 - accuracy: 0.5248 - val_loss: 0.0855 - val_accuracy: 0.5337
Epoch 6/10
678/678 [==============================] - 169s 249ms/step - loss: 0.0843 - accuracy: 0.5404 - val_loss: 0.0835 - val_accuracy: 0.5413
Epoch 7/10
678/678 [==============================] - 169s 249ms/step - loss: 0.0822 - accuracy: 0.5502 - val_loss: 0.0821 -

In [12]:
import pickle
with open('/content/drive/MyDrive/Genre Classification Dataset/lstm4.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Genre Classification Dataset/lstm3.pkl', 'rb') as f:
  model=pickle.load(f)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='micro')
recall = recall_score(y_test, y_pred_classes, average='micro')
f1 = f1_score(y_test, y_pred_classes, average='micro')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

339/339 [==============================] - 13s 37ms/step
Accuracy: 0.4260813428018076
Precision: 0.6888034314450525
Recall: 0.42949368256017706
F1-score: 0.5290842990229493


In [ ]:
# Identify misclassified instances
misclassified = np.where(np.any(y_test != y_pred_classes, axis=1))[0]

# Analyze misclassified instances and suggest improvements
for idx in misclassified[:50]:
    print(f"Movie: {df['title'][idx]}")
    print(f"Actual genres: {', '.join(df['genre'][idx])}")
    print(f"Predicted genres: {', '.join(mlb.classes_[y_pred_classes[idx] == 1])}")
    print()
print(len(y_pred_classes),len(misclassified))